In [8]:
from os.path import join, split
import pandas as pd
import os, sys
base_dir = split(os.getcwd())[0]
if base_dir not in sys.path:
    sys.path.append(base_dir)
import pickle
from utilities import utils

In [4]:
table_dir = join(base_dir, 'data','tables')

In [12]:
with open(join(table_dir, 'sids_2019.pkl'), 'rb') as f:
    sids_2019_ls = pickle.load(f)
df_clean = utils.load_scan_csv(join(table_dir, 'neg_pos_clean.csv')).dropna(how='all')
df_all = utils.load_scan_csv(join(table_dir, 'neg_pos.csv')).dropna(how='all')

C:\Users\kiril\.conda\envs\cobra\lib\site-packages\ipykernel_launcher.py:3: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\kiril\.conda\envs\cobra\lib\site-packages\ipykernel_launcher.py:4: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.


## Add 2019 tag to df_all

In [23]:
df_all['2019'] = 0
mask_2019 = df_all.SeriesInstanceUID.isin(sids_2019_ls)
df_all.loc[mask_2019, '2019'] = 1

## Drop obsolete columns

In [28]:
df_all = df_all.drop(columns=['Sequence', 'TrueSequenceType', 'Positive'])

## Compare keys, what is missing?

In [19]:
print(set(df_clean.keys()).difference(set((df_all.keys()))))

{'RowSpacing', 'ColumnSpacing', 'DistanceBetweenSlices', 'days_since_test', 'NumberOfSlices'}


## Add RowSpacing and ColumnSpacing to df_all

In [20]:
def extract_from_arr(arr, pos): 
    try:
        return arr[pos] 
    except:
        return None
df_all['RowSpacing'] = df_all.PixelSpacing.map(lambda x: extract_from_arr(x, 0))
df_all['ColumnSpacing'] = df_all.PixelSpacing.map(lambda x: extract_from_arr(x, 1))

## Add DistanceBetweenSlices, NumberOfSlices and drop days_since_test, we can later compute it on CR

In [44]:
df_final = pd.merge(df_all, 
    df_clean[['SeriesInstanceUID', 'DistanceBetweenSlices', 'NumberOfSlices']], 
    how='left', on='SeriesInstanceUID')

In [55]:
df_final.to_pickle(f"{table_dir}/scan_final.pkl")

In [46]:
df_final.to_csv(f"{table_dir}/scan_final.csv", index=False, header=True)

320556